<a href="https://colab.research.google.com/github/sibam23/memes_vs_note/blob/master/Memes_VS_Notes_Numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [386]:
import os
import numpy as np
from numpy import argmax
import cv2
from matplotlib import pyplot as plt
import random
np.random.seed(1)
import math

In [362]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [363]:
data_meme = "/content/gdrive/My Drive/Stage_0/Stage_0/Memes"
data_note = "/content/gdrive/My Drive/Stage_0/Stage_0/Notes"

In [364]:
def dataloader(folder):
  images = []
  for file in os.listdir(folder):
    img = cv2.imread(os.path.join(folder,file))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(64,64))
    images.append(img)
    
  return images

In [365]:
memes = dataloader(data_meme)
notes = dataloader(data_note)
memes = np.array(memes)
memes = memes.reshape(800,64,64,3)
memelabel = np.ones(memes.shape[0])
notes = np.array(notes)
notes = notes.reshape(800,64,64,3)
notelabel = np.zeros(notes.shape[0])
print(notelabel.shape)


(800,)


In [366]:
memedata = list(zip(memes,memelabel))
notedata = list(zip(notes,notelabel))
datatrain = []
datatrain = memedata[0:750] + notedata[0:750]
random.shuffle(datatrain)
X_train,Y_train = list(zip(*datatrain))

datatest = []
datatest = memedata[750:775] + notedata[750:775]
random.shuffle(datatest)
X_test,Y_test = list(zip(*datatest))

dataval = []
dataval = memedata[775:] + notedata[775:]
random.shuffle(dataval)
X_val,Y_val = list(zip(*dataval))


X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)

X_train = X_train.reshape(1500,-1)
X_test = X_test.reshape(50,-1)
X_val = X_val.reshape(50,-1)

X_train = np.transpose(X_train)
X_test = np.transpose(X_test)
X_val = np.transpose(X_val)
X_train.astype(float)
X_test.astype(float)
X_val.astype(float)



X_train= X_train/255.0
X_test= X_test/255.0
X_val= X_val/255.0
Y_train = np.array(Y_train)
Y_train = Y_train.reshape(1,1500)
Y_test = np.array(Y_test)
Y_test = Y_test.reshape(1,50)
Y_val = np.array(Y_val)
Y_val = Y_val.reshape(1,50)

print(Y_test.shape)


(1, 50)


In [367]:
def initialize_parameters_deep(layer_dims):
    
    
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims)            

    for l in range(1, L):
        
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
      

        
    return parameters

In [368]:
def forward_step(A_prev, W, b,method="relu"):
    
    Z = np.dot(W,A_prev) + b
    eps=1e-8
    
    if method=="sigmoid":
        A = (1/(1+np.exp(-Z)))+eps
    else:
        A=np.maximum(float(0),Z)
    cache_of_layer = (A_prev, W, b,Z)
    return A, cache_of_layer

In [369]:
def forward_propagation(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2                  
    
    for l in range(1, L):
        A_prev = A 
        W=parameters['W' + str(l)]
        b=parameters['b' + str(l)]
        A, cache = forward_step(A_prev, W, b, "relu")
        caches.append(cache)
    
    W=parameters['W' + str(L)]
    b=parameters['b' + str(L)]
    AL, cache = forward_step(A, W, b, "sigmoid")
    caches.append(cache)
    return AL, caches

In [389]:
def compute_cost(AL, Y):
     
    m = Y.shape[1]
    eps = 1e-10
 
    cost = (-1/m)*np.sum((np.multiply(np.log(AL+eps),Y))+np.multiply(np.log(1-AL+eps),(1-Y)))

    
    cost = np.squeeze(cost)

     

    
    return cost
    

In [371]:
def backward_step(dA, cache, activation):
    
    A_prev, W, b,Z = cache
    m =A_prev.shape[1]
    if activation == "relu":
        relu_derv = (Z>float(0)).astype(float)
        dZ = dA*relu_derv
        
    elif activation == "sigmoid":
        sigmoid_derv = Z*(1-Z)
        dZ = dA*sigmoid_derv
    dW = np.dot(dZ,A_prev.T)/m
    db = np.sum(dZ,axis=1,keepdims=True)/m
    dA_prev = np.dot(W.T,dZ)
    
    return dA_prev, dW, db



In [372]:
def backward_prop(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) 
    eps=1e-8
    
    dAL = - (np.divide(Y, AL+eps) - np.divide(1 - Y, ((1 - AL)+eps)))
    
    current_caches=  caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = backward_step(dAL, current_caches, activation="sigmoid")
    
    for l in reversed(range(L-1)):
        current_caches = caches[l]
        grads["dA" + str(l)], grads["dW" + str(l + 1)], grads["db" + str(l + 1)] = backward_step(grads["dA"+str(l+1)],current_caches, activation="relu")
    return grads

In [373]:

def update_parameters(parameters, grads, learning_rate):
  
    
    L = len(parameters) // 2 

    
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*grads["db"+str(l+1)]
    
    return parameters

In [374]:
def accuracy(predicts,labels):
    m = labels.shape[1]
    pred = (predicts>0.5).astype(int)
    acc = np.sum(pred==labels)
    acc= 100*acc/m
    
    
    return acc

In [405]:
def model(X,Y,layer_dims,learning_rate=0.000315,epochs=10):
    costs = []
    
           
    
    parameters = initialize_parameters_deep(layer_dims)
    
    minibatches = X.shape[1]/50
    minibatch_size= 50
    AL = np.zeros([1,Y.shape[1]])
    for i in range(epochs):
        totcost = 0
        for k in range(int(minibatches)) :
            minibatch_X = X[:,(minibatch_size*k):(minibatch_size*(k+1))]
            minibatch_Y = Y[:,(minibatch_size*k):(minibatch_size*(k+1))]
            AL[:,(minibatch_size*k):(minibatch_size*(k+1))], caches = forward_propagation(minibatch_X, parameters)
            totcost += compute_cost(AL[:,(minibatch_size*k):(minibatch_size*(k+1))], minibatch_Y)
            grads = backward_prop(AL[:,(minibatch_size*k):(minibatch_size*(k+1))],minibatch_Y, caches)
            
            
            parameters = update_parameters(parameters, grads,learning_rate)
        Cost= totcost/minibatches   
        print ("Cost after epoch %i: %f" %(i, Cost))
    
    
    return parameters,AL
    

In [403]:
 parameters,AL = model(X_train,Y_train,layer_dims=[12288,2500,1])

Cost after epoch 0: 0.674069
Cost after epoch 1: 0.662602
Cost after epoch 2: 0.651094
Cost after epoch 3: 0.639808
Cost after epoch 4: 0.629769
Cost after epoch 5: 0.621350
Cost after epoch 6: 0.614484
Cost after epoch 7: 0.608927
Cost after epoch 8: 0.604405
Cost after epoch 9: 0.600661


In [404]:
acc = accuracy(AL,Y_train)
ALt,_ = forward_propagation(X_test,parameters)
acct  = accuracy(ALt,Y_test)
ALv,_ = forward_propagation(X_val,parameters)
accv  = accuracy(ALv,Y_val)
print("train accuracy = {}".format(accu))
print("test accuracy = {}".format(acct))
print("val accuracy = {}".format(accv))




train accuracy = 62.0
test accuracy = 74.0
val accuracy = 64.0
